### 基于langchain框架实现基础的RAG流程

#### 文件的读取与切分

In [1]:
# 文件格式包括 pdf word excel
from langchain.document_loaders import PyPDFLoader, Docx2txtLoader, UnstructuredFileLoader, TextLoader

# 读取文件，可以读取网络或者本地文件
loader = PyPDFLoader("./data/tensorRT基础.pdf") 

# 属性：
# file_path: 存储PDF文件的路径。
print(loader.file_path)
# metadata: 可能包含与文档相关的元数据。
print(loader.headers)

./data/tensorRT基础.pdf
None


In [2]:
# 方法：
# load(): 读取PDF内容并返回文档对象。
# get_metadata(): 获取文档的元数据。
pages = loader.load_and_split()
# print(loader.load()[1])# 表格识别失败 (直接就不给我识别)
# print(loader.load()[2])# 顺序格式识别错误
# print(loader.load()[3])# 不处理图片(截图)
# print(loader.load()[4])# 部分代码框也没有识别出来

In [3]:
# 切分 chunk
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap  = 50,
)

docs = text_splitter.split_documents(pages)

len(docs)

38

In [4]:
print(docs[0])

page_content='TensorRT 基础  
TensorRT 基 础
一 、 准 备 知 识
1.1 环 境 配 置
A. CUDA Driver
B. CUDA
C. cuDNN
D. TensorRT
1.2  TensorRT 介 绍
二 、 构 建 阶 段
2.1 创 建 网 络 定 义
2.2 配 置 参 数
2.3 生 成 Engine
2.4 保 存 为 模 型 文 件
2.5 释 放 资 源
三 、 运 行 时 阶 段
3.1 反 序 列 化 并 创 建 Engine
3.2 创 建 一 个ExecutionContext
3.3 为 推 理 填 充 输 入
3.4 调 用 enqueueV2 来 执 行 推 理
3.5 释 放 资 源
四 、 编 译 和 运 行
一、准备知识  
NVIDIA® TensorRT™ 是一个用于高性能深度学习的推理框架。它可以与 TensorFlow 、 PyTorch 和
MXNet 等训练框架相辅相成地工作。
1.1 环境配置  
A. CUDA Driver  
使用 CUDA 前，要求 GPU 驱动与cuda 的版本要匹配，匹配关系如下：' metadata={'source': './data/tensorRT基础.pdf', 'page': 0}


In [5]:
# 切分 chunk
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,                           # 切分大小 这两个参数设置的依据是什么
    chunk_overlap  = 50,                        # 切分重叠
)

docs = text_splitter.split_documents(pages)

len(docs)

38

In [6]:
docs[20]# 需要对数据进行清洗，有些读取切分出来的完全没有意义 这里有两种优化方式 1 chunk可以根据语意进行切割 2使用大模型对文档内容进行理解精简之后再存储

Document(metadata={'source': './data/tensorRT基础.pdf', 'page': 7}, page_content='ii  tensorrt \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa08.5.3.1-1+cuda11.8 \xa0 \xa0 \n\xa0 \xa0 \xa0 \xa0 \xa0 \xa0  amd64 \xa0 \xa0 \xa0  Meta package for TensorRT')

#### embedding模型

In [7]:
# 将文字转成编码
from langchain.embeddings.openai import OpenAIEmbeddings
from dotenv import load_dotenv, find_dotenv

# 加载.env文件获取API-key
load_dotenv(find_dotenv())

embed_model = OpenAIEmbeddings()

/Users/dummy/anaconda3/envs/pytorch/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


#### 数据库

数据库选用chromdb 

数据使用data文件夹中的不同格式的文件

In [8]:
from langchain.vectorstores import Chroma

vector_store = Chroma.from_documents(documents=docs, embedding=embed_model,
                                     collection_name="my_collection",persist_directory="./directory") # 
# 持久化到磁盘
vector_store.persist()

/Users/dummy/anaconda3/envs/pytorch/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(


In [1]:
# 向量数据集的读取
from langchain.vectorstores import Chroma

# 将文字转成编码
from langchain.embeddings.openai import OpenAIEmbeddings
from dotenv import load_dotenv, find_dotenv

# 加载.env文件获取API-key
load_dotenv(find_dotenv())

embed_model = OpenAIEmbeddings()
# 加载已保存的向量数据库
loaded_vector_store = Chroma(
    collection_name="my_collection",
    persist_directory="./directory",
    embedding_function=embed_model  # 重要：提供嵌入模型
)
# 追加新的文档
# loaded_vector_store.add_documents(new_documents)

/Users/dummy/anaconda3/envs/pytorch/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(
/Users/dummy/anaconda3/envs/pytorch/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 0.4. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  warn_deprecated(


In [2]:
# 获取数据库中的部分数据
data = loaded_vector_store.get()
print(data['documents'][0])  # 打印文档内容
print(data['metadatas'])  # 打印元数据

提高了模型的适应性。
7. 集成 TensorFlow 和 TensorFlow Lite ： TensorRT 可以与 TensorFlow 和 TensorFlow Lite 集
成，使得在 NVIDIA GPU 上执行 TensorFlow 模型变得更为高效。
8. 插件支持：  TensorRT 提供了插件支持，可以通过插件自定义层的实现，以适应一些特殊的网
络结构。
TensorRT 可以在边缘设备、嵌入式系统和数据中心等场景中使用，以加速深度学习模型的推理任
务。在需要实时性能和低延迟的应用中， TensorRT 是一个强大的工具，可用于优化和部署深度学
习模型。
 
TensorRT 分两个阶段运行
构建（Build ）阶段：你向 TensorRT 提供一个模型定义， TensorRT 为目标 GPU 优化这个模型。这
个过程可以离线运行。
[{'page': 8, 'source': './data/tensorRT基础.pdf'}, {'page': 2, 'source': './data/tensorRT基础.pdf'}, {'page': 12, 'source': './data/tensorRT基础.pdf'}, {'page': 13, 'source': './data/tensorRT基础.pdf'}, {'page': 7, 'source': './data/tensorRT基础.pdf'}, {'page': 6, 'source': './data/tensorRT基础.pdf'}, {'page': 10, 'source': './data/tensorRT基础.pdf'}, {'page': 11, 'source': './data/tensorRT基础.pdf'}, {'page': 7, 'source': './data/tensorRT基础.pdf'}, {'page': 6, 'source': './data/tensorRT基础.pdf'}, {'page': 5, 'source': './data/tensorRT基础.pdf'}, {'page': 7, 'source': './data/tensorRT基础.pdf'}, {'page': 3, 'source': './data/tensor

#### 查询数据库

In [3]:
# 查询数据库
query = "添加相关 Builder 的配置。"
results = loaded_vector_store.similarity_search(query,k=2)

# 显示结果
for result in results:
    print(result.page_content)  # 打印文档内容
    print('-'*30)
    # print(result.metadata)  # 打印元数据


# 这里的优化思路是rerank
# 重排序（Re-Rank）：对检索出的文档进行重新排序，以确保最相关的文档排在前面。

TensorRT 的网络定义不会复制参数数组（如卷积的权重）。因此，在构建阶段完成之前，你不能释放这
些数组的内存。
2.2 配置参数  
下面我们来添加相关 Builder 的配置。 createBuilderConfig 接口被用来指定 TensorRT 应该如何优
化模型。如下：
在可用的配置选项中，你可以控制 TensorRT 降低计算精度的能力，控制内存和运行时执行速度之间的权
衡，并限制 CUDA® 内核的选择。由于构建器的运行可能需要几分钟或更长时间，你也可以控制构建器如
何搜索内核，以及缓存搜索结果以用于后续运行。在我们的示例代码中，我们仅配置
workspace （workspace 就是  tensorrt 里面算子可用的内存空间  ）大小和运行时 batch size ，如
下：
2.3 生成 Engine  
在你有了网络定义和 Builder 配置后，你可以调用 Builder 来创建Engine 。Builder 以一种称为
plan 的序列化形式创建 Engine ，它可以立即反序列化，也可以保存到磁盘上供以后使用。需要注意的
------------------------------
构建阶段的最高级别接口是  Builder 。Builder 负责优化一个模型，并产生 Engine 。通过如下接口创
建一个Builder 。
要生成一个可以进行推理的 Engine ，一般需要以下三个步骤：
创建一个网络定义
填写Builder 构建配置参数，告诉构建器应该如何优化模型
调用Builder 生成Engine
2.1 创建网络定义  
NetworkDefinition 接口被用来定义模型。如下所示：
接口createNetworkV2 接受配置参数，参数用按位标记的方式传入。比如上面激活 explicitBatch ，
是通过1U << static_cast<uint32_t>
(nvinfer1::NetworkDefinitionCreationFlag::kEXPLICIT_BATCH); 将explicitBatch 对应的配置位
设置为 1 实现的。在新版本中，请使用 createNetworkV2 而非其他任何创建 NetworkDefinition 的接
口。
----------------------------

In [12]:
# 总结下来这个数据库用着不舒服 想要有个带ui界面可视化的那种


#### 构建prompt

In [1]:
# 构建prompt的时候也有几点优化
# 比如query优化

from langchain.vectorstores import Chroma

# 将文字转成编码
from langchain.embeddings.openai import OpenAIEmbeddings
from dotenv import load_dotenv, find_dotenv

# 加载.env文件获取API-key
load_dotenv(find_dotenv())

embed_model = OpenAIEmbeddings()
# 加载已保存的向量数据库
loaded_vector_store = Chroma(
    collection_name="my_collection",
    persist_directory="./directory",
    embedding_function=embed_model  # 重要：提供嵌入模型
)
# 追加新的文档

# loaded_vector_store.add_documents(new_documents)


/Users/dummy/anaconda3/envs/pytorch/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(
/Users/dummy/anaconda3/envs/pytorch/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 0.4. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  warn_deprecated(


In [4]:
def augment_prompt(query,loaded_vector_store):
    # 从向量数据库中获取top3的文本片段
    result = loaded_vector_store.similarity_search(query,k=3)
    
    source_knowleage = '/n'.join([x.page_content for x in result])
    # 构建prompt
    augment_prompt = f"""
    使用以下信息，回答问题：
    信息：
    {source_knowleage}
    问题：
    {query}
    """
    return augment_prompt
prompt = augment_prompt("如何添加相关Builder",loaded_vector_store)

In [6]:
print(prompt)


    使用以下信息，回答问题：
    信息：
    构建阶段的最高级别接口是  Builder 。Builder 负责优化一个模型，并产生 Engine 。通过如下接口创
建一个Builder 。
要生成一个可以进行推理的 Engine ，一般需要以下三个步骤：
创建一个网络定义
填写Builder 构建配置参数，告诉构建器应该如何优化模型
调用Builder 生成Engine
2.1 创建网络定义  
NetworkDefinition 接口被用来定义模型。如下所示：
接口createNetworkV2 接受配置参数，参数用按位标记的方式传入。比如上面激活 explicitBatch ，
是通过1U << static_cast<uint32_t>
(nvinfer1::NetworkDefinitionCreationFlag::kEXPLICIT_BATCH); 将explicitBatch 对应的配置位
设置为 1 实现的。在新版本中，请使用 createNetworkV2 而非其他任何创建 NetworkDefinition 的接
口。/nTensorRT 的网络定义不会复制参数数组（如卷积的权重）。因此，在构建阶段完成之前，你不能释放这
些数组的内存。
2.2 配置参数  
下面我们来添加相关 Builder 的配置。 createBuilderConfig 接口被用来指定 TensorRT 应该如何优
化模型。如下：
在可用的配置选项中，你可以控制 TensorRT 降低计算精度的能力，控制内存和运行时执行速度之间的权
衡，并限制 CUDA® 内核的选择。由于构建器的运行可能需要几分钟或更长时间，你也可以控制构建器如
何搜索内核，以及缓存搜索结果以用于后续运行。在我们的示例代码中，我们仅配置
workspace （workspace 就是  tensorrt 里面算子可用的内存空间  ）大小和运行时 batch size ，如
下：
2.3 生成 Engine  
在你有了网络定义和 Builder 配置后，你可以调用 Builder 来创建Engine 。Builder 以一种称为
plan 的序列化形式创建 Engine ，它可以立即反序列化，也可以保存到磁盘上供以后使用。需要注意的/n口。
将模型转移到 TensorRT 的最常见的方式是

#### 调用部署好的大模型进行测试

In [7]:
# 利用transform库进行模型加载及推理实验
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/Qwen2.5-0.5B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [17]:
prompt = augment_prompt("什么是Builder",loaded_vector_store)
messages = [
    {"role": "system", "content": "你是千问，一个由阿里云开发的大语言模型"},
    {"role": "user", "content": prompt},
]

In [18]:
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
)

In [19]:
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512,
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

In [20]:
print(response)

Builder 是 TensorFlow 的一个组件，负责优化和创建机器学习模型。它是一个抽象类，包含一系列方法，帮助用户创建和管理复杂的模型架构。

以下是 `Builder` 类的一些关键特性：

1. **优化功能**：Builder 提供了多种优化策略，包括自动微分、批量归一化、动态重叠等，可以帮助用户快速构建高性能的模型。

2. **创建网络定义**：`createNetworkV2` 接口允许开发者定义模型的结构，例如是否启用显式批处理、特定网络的框架选择等。

3. **配置参数**：Builder 支持通过配置参数定制模型，包括计算精度、内存管理和运行时性能等方面的设定。

4. **计划与反序列化**：Builder 可以创建引擎并立即反序列化，或者将其保存到磁盘以便于后续使用。

5. **生命周期管理**：Builder 允许用户根据需要随时清理资源，确保资源的有效利用。

6. **支持不同优化技术**：Builder 支持多种优化技术，包括手动微分、批量归一化等，用户可以根据自己的需求选择合适的优化方案。

7. **跨平台兼容性**：Builder 旨在提供一个通用的方法来创建和优化模型，适用于各种设备和架构。

总之，Builder 是 TensorFlow 中的一个核心工具，提供了强大的优化能力，使得用户能够快速构建高性能的模型。


In [16]:
print(prompt)


    使用以下信息，回答问题：
    信息：
    构建阶段的最高级别接口是  Builder 。Builder 负责优化一个模型，并产生 Engine 。通过如下接口创
建一个Builder 。
要生成一个可以进行推理的 Engine ，一般需要以下三个步骤：
创建一个网络定义
填写Builder 构建配置参数，告诉构建器应该如何优化模型
调用Builder 生成Engine
2.1 创建网络定义  
NetworkDefinition 接口被用来定义模型。如下所示：
接口createNetworkV2 接受配置参数，参数用按位标记的方式传入。比如上面激活 explicitBatch ，
是通过1U << static_cast<uint32_t>
(nvinfer1::NetworkDefinitionCreationFlag::kEXPLICIT_BATCH); 将explicitBatch 对应的配置位
设置为 1 实现的。在新版本中，请使用 createNetworkV2 而非其他任何创建 NetworkDefinition 的接
口。/nTensorRT 的网络定义不会复制参数数组（如卷积的权重）。因此，在构建阶段完成之前，你不能释放这
些数组的内存。
2.2 配置参数  
下面我们来添加相关 Builder 的配置。 createBuilderConfig 接口被用来指定 TensorRT 应该如何优
化模型。如下：
在可用的配置选项中，你可以控制 TensorRT 降低计算精度的能力，控制内存和运行时执行速度之间的权
衡，并限制 CUDA® 内核的选择。由于构建器的运行可能需要几分钟或更长时间，你也可以控制构建器如
何搜索内核，以及缓存搜索结果以用于后续运行。在我们的示例代码中，我们仅配置
workspace （workspace 就是  tensorrt 里面算子可用的内存空间  ）大小和运行时 batch size ，如
下：
2.3 生成 Engine  
在你有了网络定义和 Builder 配置后，你可以调用 Builder 来创建Engine 。Builder 以一种称为
plan 的序列化形式创建 Engine ，它可以立即反序列化，也可以保存到磁盘上供以后使用。需要注意的/n口。
将模型转移到 TensorRT 的最常见的方式是

In [21]:
# 测试结果其实并不是很好 一方面是因为query和context的关系并不大 所以我们是否要考虑不使用 向量数据库 尝试使用知识图谱